In [ ]:
import pandas as pd
import seaborn as sns
from colorama import Fore
import matplotlib.pyplot as plt
import numpy as np
import re
import scikitplot as skplt 

from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import xgboost as xgb

from IPython.display import Image
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [ ]:
def report(y_true, y_pred, labels):
    f1 = f1_score(y_true=y_true, y_pred=y_pred, labels=labels, average='macro', pos_label=1)
    
    acc = accuracy_score(y_true=y_true, y_pred=y_pred)
    
    cm = pd.DataFrame(confusion_matrix(y_true=y_true, 
                                       y_pred=y_pred), 
                      index=labels, 
                      columns=labels)
    rep = classification_report(y_true=y_true, 
                                y_pred=y_pred)
    
    return (f'F1 Macro = {f1:.4f}\n\nAccuracy = {acc:.4f}\n\nConfusion Matrix:\n{cm}\n\nClassification Report:\n{rep}')

# Poverty Level Prediction of Costa Rican Households

# Outline:

1. Introduction
2. Data Cleaning
3. Base Model
4. Adding Features
5. Visualization
6. Feature Selection
7. Models

# 1. Introduction:

* "Many social programs have a hard time making sure the right people are given enough aid. It’s especially tricky when a program focuses on the poorest segment of the population. The world’s poorest typically can’t provide the necessary income and expense records to prove that they qualify.
* In Latin America, one popular method uses an algorithm to verify income qualification. It’s called the Proxy Means Test (or PMT). With PMT, agencies use a model that considers a family’s observable household attributes like the material of their walls and ceiling, or the assets found in the home to classify them and predict their level of need.

* While this is an improvement, accuracy remains a problem as the region’s population grows and poverty declines". 
* Source: https://www.kaggle.com/c/costa-rican-household-poverty-prediction/data

In [ ]:
data = pd.read_csv('../input/train.csv', index_col='idhogar')
data = data.rename(columns = {'v2a1':'rent','hacdor':'crowd_bedroom', 'hacapo': 'crowd_room', 'v14a': 'bathroom' })
print(data.shape)
data.tail(7)

* Each row represents one family member.
* Each row contains individual features and household features.
* According to kaggles file description, only predictions for heads of household are scored.

In [ ]:
data.info(verbose=False)

In [ ]:
data.describe()

### Target Variable: Poverty level
1. <u>extreme poverty</u> 
2. moderate poverty 
3. vulnerable households 
4. non vulnerable households

The main class I would like to identify and predict is the extreme poverty households - class 1. 

In [ ]:
data = data.reset_index()
data.Target.value_counts(normalize=True)

* The data is imbalanced

The evaluation is based on the <b> F1 macro score</b>, which is the <b>harmonic mean of precision and recall</b>.

# 2. Data Cleaning

### Dropping duplicated features:

In [ ]:
print((data['agesq'] == data['SQBage']).all())
print((data['tamhog'] == data['hhsize']).all())
print((data['hhsize'] == data['hogar_total']).all())
data.drop(columns=['agesq', 'tamhog','hhsize'], inplace=True)
data.shape

### Creating Ordinal Features from dummies:

In [ ]:
issues_list = ['epared', 'etecho', 'elimbasu', 'eviv', 'tipovivi', 'area'] #good, normal, bad etc.

for issue in issues_list:

    col_list = list(data.columns[data.columns.str.startswith(issue)])
    df = data[col_list]
    
    s = pd.Series(df.columns[np.where(df!=0)[1]])
    s = s.apply(lambda x: x[-1]).astype('int64')
    s = s.rename(issue+'_cat', copy=True)
    
    data = pd.concat([data, s], axis=1)
    data = data.drop(columns=col_list)
data.head()

In [ ]:
data.shape

In [ ]:
#educational level

def instlevel_cat(row):
    col_list = list(data.columns[data.columns.str.startswith('instlevel')])
    
    for i in col_list:
        if row[i] == 1:
            return (int(i[-1]))

In [ ]:
# water provision

def water_cat(row):
    if row.abastaguadentro == 1:
        return 1
    if row.abastaguafuera == 1:
        return 2
    if row.abastaguano == 1:
        return 3

# source of energy used for cooking
def energcocinar_cat(row):
    if row.energcocinar1 == 1:
        return 1
    if row.energcocinar4 == 1:
        return 2
    if row.energcocinar3 == 1:
        return 3
    if row.energcocinar2 == 1:
        return 4

In [ ]:
data['water_cat'] = data.apply(water_cat, axis=1)
data['energcocinar_cat'] = data.apply(energcocinar_cat, axis=1)
data['instlevel_cat'] = data.apply(instlevel_cat, axis=1)

data.shape

Before dropping these categorical features, I found that there are null values in instlevel_cat:

In [ ]:
print('nulls: ', data['instlevel_cat'].isnull().sum())

In [ ]:
data[data['instlevel_cat'].isnull()]['escolari']

In [ ]:
data.instlevel_cat.fillna(1, inplace=True)

* Dropping these dummies:

In [ ]:
import re

columns_list = list(data.columns)
col_drop = []

for i in columns_list:
    if re.match("instlevel\d", i) or re.match("abastagua", i) or re.match("energcocinar\d", i):
        col_drop.append(i)
print(col_drop)

In [ ]:
print('Before:',data.shape)
data.drop(columns=col_drop, inplace=True)
print('After:',data.shape)

In [ ]:
data[['female', 'male']].head(3)

In [ ]:
data = data.rename(columns={'female': 'sex'}).drop(columns='male')
data.shape

### Fixing mixed Scales:

In [ ]:
data[['dependency','edjefe','edjefa']].head(5)


In [ ]:
for col in ['dependency','edjefe','edjefa']:
    print(col.capitalize(),':\n', data[col].unique(),'\n')
    

In [ ]:
for col in ['dependency','edjefe','edjefa']:
    data[col].replace({'yes': 1, 'no': 0 }, inplace=True)
    data[col] = data[col].astype('float64')
    if data[col].dtype == 'float64':
        print(col,': fixed')

In [ ]:
columns_list = list(data.columns)
print(len(columns_list))
print(columns_list)

### Dropping all squared parameters:
* I will create new ones without null values or calculation mistakes

In [ ]:
print('Before:',data.shape)

col_drop = []

for i in columns_list:
    if re.match("SQB", i):
        col_drop.append(i)
print(col_drop)

data.drop(columns=col_drop, inplace=True)

print('After:',data.shape)

### Filling Null Values:

In [ ]:
data.isnull().sum().sort_values(ascending=False).head(7)

### rez_esc - Years behind in school

In [ ]:
data.rez_esc.value_counts()

In [ ]:
ages = list(data[data['rez_esc'].isnull()]['age'].unique())
print(sorted(ages))

This feature is available between the ages of 11-17

In [ ]:
data[['rez_esc', 'age']].head()

In [ ]:
data['rez_esc'] = data['rez_esc'].fillna(0)

In [ ]:
data.isnull().sum().sort_values(ascending=False).head()

### v18q1 - Number of tablets in the household

In [ ]:
data.v18q1.describe(np.arange(0,1.1,0.2))

In [ ]:
data.v18q1.value_counts()

The category 0 doesn't exist, so I checked it with the boolean feature v18q:

In [ ]:
data[['v18q1', 'v18q']].head()

In [ ]:
print(data[data['v18q1'].isnull()]['v18q'].sum())
data['v18q1'] = data['v18q1'].fillna(0)

In [ ]:
data.isnull().sum().sort_values(ascending=False).head()

### rent - house rental fees 

In [ ]:
data[['rent', 'tipovivi_cat']].head() #own the house or no house to pay rent for

In [ ]:
data[data['rent'].isnull()]['tipovivi_cat'].unique()

There are families who own the house or does't have one, so there are no rental fees.

In [ ]:
data['rent'] = data['rent'].fillna(0)

In [ ]:
data.isnull().sum().sort_values(ascending=False).head()

### meaneduc - Average years of education for adults (18+)

In [ ]:
meaneduc_df = data.groupby(['idhogar', 'Id'])['age', 'escolari'].sum()
meaneduc_df.head()

In [ ]:
def fix_meaneduc(df): 
    counter=0
    escolari=0
    
    list_age = list(df.age) 
    list_escolari = list(df.escolari) 
    
    for i in range(len(list_age)):
        if (list_age[i] >= 18):
            counter +=1
            escolari += list_escolari[i]
    if counter==0:
        return 0
    return (escolari/counter)
    

In [ ]:
meaneduc_new = data.groupby('idhogar').apply(fix_meaneduc)
meaneduc_new.isnull().any()


In [ ]:
meaneduc_new.head() #series, each row is for idhogar

In [ ]:
print(meaneduc_new.shape) 
print((data.parentesco1==1).sum()) 

In [ ]:
data[data['meaneduc'].isnull()]

Since I would like to create a new table where each row represents a household, I will drop meaneduc and replace it with meaneduc_new later on.

# 3. Houseowners Model

### Creating a dataset of houseowners (where parentesco1==1)
* This dataset includes houseowners individual features and house features.

In [ ]:
# family example:
print(data.shape)
data[data['idhogar'] == '2b58d945f']

* Dropping meaneduc - family feature
* Dropping columns that repeat themselves in a boolean and ordinal manner ('mobilephone', 'v18q').

In [ ]:
data = data.drop(columns=['meaneduc','mobilephone', 'v18q'])
data.shape

In [ ]:
# Houseowners Example:
owners_data = data[data['parentesco1']==1]
print(owners_data.shape)
owners_data.head()

### First Model: Houseowners' features only

I will compare the next models to this model, in order to understand if we actually need the features of the family or not.

In [ ]:
train, test = split(owners_data, test_size=0.3, random_state=44, stratify=owners_data.Target)
train.shape, test.shape

In [ ]:
train.Target.value_counts(normalize=True)

### Imbalances data - I will use the balanced class_weight

In [ ]:
rf = RandomForestClassifier(class_weight='balanced', n_estimators=300, 
                            max_depth=30, min_samples_leaf=10, max_features=30, random_state=111)

X = train.drop(columns=['Target','Id','idhogar'])
y = train.Target

params = {'max_depth': range(25,30),
          'min_samples_leaf': range(7,10)}

gs = GridSearchCV(rf, params, cv=5, scoring='f1_macro')
gs.fit(X,y)

In [ ]:
model1 = gs.best_estimator_
gs.best_params_

In [ ]:
X_test = test.drop(columns=['Target','Id','idhogar'])
y_test = test.Target

print(Fore.BLUE+report(y_test, model1.predict(X_test), model1.classes_))


In [ ]:
my_list = list(zip(model1.feature_importances_ ,X.columns))
my_list.sort(key=lambda tup: tup[0],reverse=True)
# for item in my_list:
#     if item[0]> 0.01:
#         print(item)

In [ ]:
importances = pd.DataFrame(data=None,columns=['importance', 'feature'])

importance_l = []
feature_l = []

for t in my_list:
    importance_l.append(t[0])
    feature_l.append(t[1])
    
importances['importance'] = importance_l
importances['feature'] = feature_l

In [ ]:
plt.figure(figsize=(20,5))
plt.xticks(rotation=90)
plt.title('Feature Importances')
sns.barplot(x="feature", y="importance", data=importances)

In [ ]:
owners_data_only = owners_data.copy()

# 4. Adding Features


### Adding meaneduc_new:

In [ ]:
# adding meaneduc_new
owners_data = owners_data.merge(meaneduc_new.to_frame(), left_on='idhogar', right_index=True)\
                        .rename(index=str, columns={0: "meaneduc_new"})

owners_data.head()

In [ ]:
print(list(data.columns))

### Creating a dataframe of individual features:

In [ ]:
# Individual Features:

family_data = data[['idhogar', 'Id','escolari', 'rez_esc','dis','instlevel_cat','age',
                           'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 
                           'estadocivil5', 'estadocivil6', 'estadocivil7', 
                           'parentesco1', 'parentesco2', 'parentesco3', 'parentesco4', 
                           'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 
                           'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12']]
print(owners_data.shape, family_data.shape)
family_data.head()

In [ ]:
family_data[family_data.idhogar == '2b58d945f']

### Dropping  family members without a houseowner:

In [ ]:
print(f'There are {(len(family_data.idhogar.unique()))-(len(owners_data.idhogar.unique()))} family members with no related homeowner')


In [ ]:
wrong_idhogars = []
for id_ in list(family_data.idhogar.unique()):
    if id_ not in owners_data.idhogar.unique():
        wrong_idhogars.append(id_)
print(wrong_idhogars)

I will drop them since the score is given to the homeowners.

### Creating family features:
* Aggregating individual features of each family member to one family feature

In [ ]:
print(family_data.shape)
for id_ in wrong_idhogars:
    family_data = family_data[family_data['idhogar'] != id_]
print('Number of family members in the dataset (rows): ',family_data.shape)

In [ ]:
print('Number of families in the dataset: ',owners_data.shape)

owners_data = owners_data.sort_values(by='idhogar')
family_data = family_data.sort_values(by='idhogar')

df = family_data.groupby('idhogar')

In [ ]:
#school min, max and mean per family

owners_data['escolari_max_fam'] = np.array(df.escolari.max())
owners_data['escolari_min_fam'] = np.array(df.escolari.min())
owners_data['escolari_mean_fam'] = np.array(df.escolari.mean())

owners_data.shape

In [ ]:
#age min, max and mean per family

owners_data['age_max_fam'] = np.array(df.age.max())
owners_data['age_min_fam'] = np.array(df.age.min())
owners_data['age_mean_fam'] = np.array(df.age.mean())
owners_data['age_median_fam'] = np.array(df.age.median())

owners_data.shape

In [ ]:
#rez_esc max and mean per family

owners_data['rez_esc_max'] = np.array(df.rez_esc.max())
owners_data['rez_esc_mean'] = np.array(df.rez_esc.mean())

owners_data.shape

In [ ]:
# sum of disabled in the family
owners_data['dis_fam'] = np.array(df.rez_esc.sum())
owners_data.shape

In [ ]:
#mean of family members that studied in each education level:
owners_data['instlevel_mean_fam'] = np.array(df.instlevel_cat.mean())
owners_data.shape

In [ ]:
col_names = pd.Series(owners_data.columns)
print(list(col_names))

In [ ]:
#sum of family members that are married, divorced etc.:

col_list = owners_data.columns[col_names.str.startswith('estadocivil')]

for col in col_list:
    owners_data[col+'_fam'] = np.array(df[col].sum())

print(owners_data.shape)
owners_data.head()

In [ ]:
#sum of family members in each family category (spouse/partner, son/doughter, mother/father etc.):

col_names = pd.Series(owners_data.columns)
#col_names
col_list = owners_data.columns[col_names.str.startswith('parentesco')]
for col in col_list:
    owners_data[col+'_fam'] = np.array(df[col].sum())

print(owners_data.shape)
owners_data.head()

In [ ]:
print(list(owners_data.columns))

### Dropping part of the individual features of houseowners:

In [ ]:
print('Before:',owners_data.shape)

owners_data.drop(columns=['Id','rez_esc',
                   'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 
                   'estadocivil5', 'estadocivil6', 'estadocivil7', 
                   'parentesco1','parentesco1_fam','parentesco2', 'parentesco3', 'parentesco4', 'parentesco5',
                   'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10',
                   'parentesco11', 'parentesco12' ], inplace=True)

print('After:',owners_data.shape)

In [ ]:
owners_data = owners_data.set_index('idhogar')
print(owners_data.shape)
owners_data.head()

### Adding Features and Interactions:

In [ ]:
owners_data_all_features = owners_data.copy()
owners_data.tail()

In [ ]:
owners_data['instlevel_cat*age_mean_fam'] = owners_data['instlevel_cat']*owners_data['age_mean_fam']
owners_data['instlevel_cat*escolari_mean_fam'] = owners_data['instlevel_cat']*owners_data['escolari_mean_fam']
owners_data['v18q1/r4t2'] = owners_data['v18q1']/owners_data['r4t2'] #tablets/people
owners_data['qmobilephone/r4t2'] = owners_data['qmobilephone'] /  owners_data['r4t2'] #phones/people

owners_data['SQescolari_mean_fam'] = np.square(owners_data['escolari_mean_fam'])
owners_data['SQdependency'] = np.square(owners_data['dependency'])
owners_data['SQmeaneduc_new'] =  np.square(owners_data['meaneduc_new'])
owners_data['SQescolari_mean_fam'] = np.square(owners_data.escolari_mean_fam)
owners_data['SQescolari'] = np.square(owners_data['escolari'])
owners_data['SQage'] = np.square(owners_data.age)
owners_data['SQhogar_total'] = np.square(owners_data['hogar_total'])
owners_data.shape

# 5. Visualization

In [ ]:
plt.figure(figsize=(25,7))
owners_data.nunique().sort_values(ascending=False).plot(kind='bar')

* most of the features are binary, the rest are ordinal or continuous

In [ ]:
corrmat = owners_data.corr()
cols = corrmat.nlargest(30, 'Target')['Target'].index
cm = np.corrcoef(owners_data[cols].values.T)
sns.set(font_scale=1.5)
plt.figure(figsize=(20, 20))

hm = sns.heatmap(cm, cbar=False, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.yticks(rotation=0)
plt.xticks(rotation=90)
plt.show()

* No feature is highly correlated with the target

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Mean Years of Education per Family by Poverty Level')
ax = sns.boxplot(x="Target", y="escolari_mean_fam", data=owners_data)

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Median age per Family by Poverty Level')
ax = sns.boxplot(x="Target", y="age_median_fam", data=owners_data)

In [ ]:
colors = {1: 'b', 2:'orange', 3: 'g', 4:'r'}
plt.figure(figsize = (12, 3))

# overcrowding - persons per room
# meaneduc_new - mean years of education of adults (>=18)

for i, col in enumerate(['overcrowding', 'meaneduc_new']):
    ax = plt.subplot(2, 1, i + 1)
    
    for poverty_level, color in colors.items():
        sns.kdeplot(owners_data.loc[owners_data['Target'] == poverty_level, col], 
                     ax = ax, color = color, label = poverty_level)
        
    plt.title(f'{col.capitalize()} Distribution'); plt.xlabel(f'{col}'); plt.ylabel('Density')
plt.subplots_adjust(top = 4)

In [ ]:
# from target, how many appeared in each instlevel

map_df = pd.crosstab(columns=owners_data.Target, 
                          index=owners_data.instlevel_cat, 
                          normalize='columns')

fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(ax=ax, data=map_df, cmap='coolwarm')

# PCA For Visualization

In [ ]:
X = owners_data.drop(columns='Target')
y = owners_data.Target

In [ ]:
X_train, X_test, y_train, y_test = split(X,y, random_state=555, test_size=0.3, stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
train_scores = []
# test_scores = []
for k in range(1, X.shape[1]):
    scaled_pca_transformer = PCA(n_components=k).fit(X_train_scaled)
        
    X_train_scaled_pca = scaled_pca_transformer.transform(X_train_scaled)
        
    clf = KNeighborsClassifier().fit(X_train_scaled_pca, y_train)
    
    X_test_scaled_pca = scaled_pca_transformer.transform(X_test_scaled)
    
    train_scores.append(clf.score(X_train_scaled_pca, y_train))
#    test_scores.append(clf.score(X_test_scaled_pca, y_test))

In [ ]:
plt.figure(figsize=(20,7))
# plt.plot(list(zip(train_scores, test_scores)), linewidth=5)
plt.plot(train_scores, linewidth=5)
plt.xticks(ticks=np.arange(0, len(X.columns)+1, 2.0))
plt.title('Model score vs. number of components')
plt.xlabel('n_components')
plt.ylabel('Score (accuracy)')
plt.legend(['train score', 'test score'], loc='best')

In [ ]:
pca_transformer = PCA(n_components=2).fit(X_train_scaled)
X_train_scaled_pca = pca_transformer.transform(X_train_scaled)
X_test_scaled_pca = pca_transformer.transform(X_test_scaled)
X_train_scaled_pca[:1]

In [ ]:
plt.figure(figsize=(15,7))
sns.scatterplot(x=X_train_scaled_pca[:, 0], 
                y=X_train_scaled_pca[:, 1], 
                hue=y_train, 
                sizes=100,
                palette="Accent") 

# 6. Feature Selection

### Variance Threshold:

In [ ]:
X = owners_data.drop(columns='Target')
y = owners_data.Target

In [ ]:
X_train, X_test, y_train, y_test = split(X, y, random_state=555, test_size=0.3, stratify=y)

In [ ]:
print('train:')
print(y_train.value_counts(normalize=True))
print('test:')
print(y_test.value_counts(normalize=True))

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) 

X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled,index=X_test.index, columns=X_test.columns)
X_train_scaled.head()

In [ ]:
selector = VarianceThreshold(0.008)
selector.fit(X_train_scaled)

In [ ]:
print(f'There are {len(X_train_scaled.columns[selector.get_support()])} columns left')

In [ ]:
dropped_features = X_train_scaled.columns[~selector.get_support()]
dropped_features

In [ ]:
selected_data_train = selector.transform(X_train_scaled)
selected_data_test = selector.transform(X_test_scaled)


selected_data_train = pd.DataFrame(selected_data_train, 
                             columns=X_train_scaled.columns[selector.get_support()])
selected_data_test = pd.DataFrame(selected_data_test, 
                             columns=X_test_scaled.columns[selector.get_support()])

selected_data_train.sample(5)

# 7. Models

### Models using PCA

* according to the PCA graph above - I chose to use 50 conponents.

In [ ]:
pca_transformer = PCA(n_components=50).fit(selected_data_train)
X_train_scaled_pca = pca_transformer.transform(selected_data_train)
X_test_scaled_pca = pca_transformer.transform(selected_data_test)

### VotingClassifier (LogisticRegression and KNeighborsClassifier)

In [ ]:
classifiers = [('LR', LogisticRegression(solver='lbfgs', multi_class='auto', 
                            max_iter=1000, class_weight='balanced',
                            random_state=555)), 
               ('KNC', KNeighborsClassifier(n_neighbors=5, metric='manhattan'))] #no class_weights


model2 = VotingClassifier(estimators=classifiers, voting='soft')
model2.fit(X_train_scaled_pca, y_train)

In [ ]:
print(Fore.BLUE+report(y_test, model2.predict(X_test_scaled_pca), model2.classes_))

## Models without using PCA

### VotingClassifier (LogisticRegression and KNeighborsClassifier)

In [ ]:
classifiers = [('LR', LogisticRegression(solver='lbfgs', multi_class='auto', 
                            max_iter=1000, class_weight='balanced',
                            random_state=555)), 
               ('KNC', KNeighborsClassifier(n_neighbors=5, metric='manhattan'))] #no class_weights


model4 = VotingClassifier(estimators=classifiers, voting='soft')
model4.fit(selected_data_train, y_train)

In [ ]:
print(Fore.BLUE+report(y_test, model4.predict(selected_data_test), model4.classes_))

### AdaBoostClassifier (DecisionTreeClassifier)

In [ ]:
dt = DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=555 )

model8 = AdaBoostClassifier(base_estimator=dt, learning_rate=0.01, n_estimators=100, random_state=222)
model8.fit(selected_data_train ,y_train)

In [ ]:
print(Fore.BLUE+report(y_test, model8.predict(selected_data_test), model8.classes_))

### BaggingClassifier (SVC):

In [ ]:
svc = SVC(class_weight='balanced',random_state=222, kernel='rbf', C= 1.0, gamma='auto')

params = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
          'C': [0.1, 1.0, 10]}

gs = GridSearchCV(svc, params, cv=3, return_train_score=False, scoring='f1_macro')
gs.fit(selected_data_train, y_train)

In [ ]:
print(gs.best_params_)
svc_best = gs.best_estimator_

In [ ]:
model10 = BaggingClassifier(base_estimator=svc_best, n_estimators=100, n_jobs=4, max_features=30, random_state=555)

model10.fit(selected_data_train, y_train)

In [ ]:
print(Fore.BLUE+report(y_test, model10.predict(selected_data_test), model10.classes_))

### LogisticRegression

In [ ]:
lr = LogisticRegression(solver='lbfgs', multi_class='auto', 
                            max_iter=10000, class_weight='balanced',
                            random_state=555)
params = {'C': [0.1, 1.0, 10]}

gs = GridSearchCV(lr, params, scoring='f1_macro', cv=5, return_train_score=False)
gs.fit(selected_data_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
model5 = gs.best_estimator_
print(Fore.BLUE+report(y_test, model5.predict(selected_data_test), model5.classes_))

In [ ]:
rf = RandomForestClassifier(class_weight='balanced', n_estimators=50000, #max_depth=30, 
                            min_samples_leaf=5, max_features=30, n_jobs=4, random_state=222)

params = {#'max_depth': range(25,30),
          'min_samples_leaf': range(5,7)}

gs = GridSearchCV(rf, params, cv=5, return_train_score=False, scoring='f1_macro')
gs.fit(selected_data_train,y_train)

In [ ]:
print(gs.best_params_)
model6 = gs.best_estimator_

### train:

In [ ]:
print(Fore.RED+report(y_train, model6.predict(selected_data_train), model6.classes_))

### test:

In [ ]:
print(Fore.BLUE+report(y_test, model6.predict(selected_data_test), model6.classes_))

### This is the best model so far, which I chose to work with.

In [ ]:
grid_results = pd.DataFrame(gs.cv_results_)
grid_results.sort_values(by='rank_test_score').head(4)

In [ ]:
df_scores = grid_results.sort_values(by='rank_test_score')[['params', 'mean_test_score', 'std_test_score' ]]

df_scores[['mean_test_score',  'std_test_score']].plot(kind='scatter', x='mean_test_score', y='std_test_score', 
                                                       color='lightblue', figsize=(10,5))

P = [df_scores.iloc[0,1] , df_scores.iloc[0,2]]
plt.plot(P[0], P[1], marker='o', markersize=5, color="darkblue")

In [ ]:
my_list = list(zip(model6.feature_importances_ ,selected_data_test.columns))
my_list.sort(key=lambda tup: tup[0],reverse=True)
# for item in my_list:
#     if item[0]> 0.009:
#         print(item)

In [ ]:
importances = pd.DataFrame(data=None,columns=['importance', 'feature'])

importance_l = []
feature_l = []

for t in my_list:
    importance_l.append(t[0])
    feature_l.append(t[1])
    
importances['importance'] = importance_l
importances['feature'] = feature_l

In [ ]:
plt.figure(figsize=(20,5))
plt.xticks(rotation=90)
plt.title('Feature Importances')
sns.barplot(x="feature", y="importance", data=importances)

### Clustering as a model:

In [ ]:
model13 = KMeans(n_clusters=4).fit(selected_data_train)
y_pred = model13.predict(selected_data_train)
print(f1_score(y_true=y_train, y_pred=y_pred, average='macro',labels=np.unique(y_pred)))

In [ ]:
print(f1_score(y_true=y_test, 
               y_pred=model13.predict(selected_data_test), 
               average='macro', 
               labels=np.unique(model13.predict(selected_data_test))))

### RandomForestClassifier with clustering as a parameter:

In [ ]:
selected_data_train1 = selected_data_train.copy()
selected_data_test1= selected_data_test.copy()

model13 = KMeans(n_clusters=4).fit(selected_data_train1)

selected_data_train1['clustering'] = model13.predict(selected_data_train1)
selected_data_test1['clustering'] = model13.predict(selected_data_test1)


In [ ]:
rf = RandomForestClassifier(class_weight='balanced', n_estimators=50000, 
                            min_samples_leaf=5, max_features=30, n_jobs=4, random_state=222)

params = {#'max_depth': range(25,30),
          'min_samples_leaf': range(5,7)}

gs = GridSearchCV(rf, params, cv=5, return_train_score=False, scoring='f1_macro')
gs.fit(selected_data_train1,y_train)

In [ ]:
print(gs.best_params_)
model14 = gs.best_estimator_

In [ ]:
print(Fore.BLUE+report(y_test, model14.predict(selected_data_test1), model14.classes_))

* Clustering doesn't improve the f1 macro score. The previous model was better.


recommendations for future research:
* Improve feature selection
* Compare models by other score methods except f1 score

# Thank you!